In [1]:
import pandas as pd
import numpy as np
import torch
from torchtext import datasets
import random
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import re
SEED=42
TEST_SIZE=0.2

In [2]:
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"



df = pd.read_csv("../data/training.1600000.processed.noemoticon.csv", encoding =DATASET_ENCODING , names=DATASET_COLUMNS)

data=pd.DataFrame()
data["text"]=df["text"]
data["target"]=df["target"]

### Equal Number of Negative and Positive Tweet

In [3]:
df.groupby("target").size()

target
0    800000
4    800000
dtype: int64

In [4]:
nltk.download('stopwords')
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shaunkim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
decode_map = {0: "NEGATIVE", 2: "NEUTRAL", 4: "POSITIVE"}

def decode_sentiment(label):
    return decode_map[int(label)]
data.target = data.target.apply(lambda x: decode_sentiment(x))

In [6]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"


def preprocess(text, stem=False):
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

data.text = data.text.apply(lambda x: preprocess(x))

In [7]:
train,test = train_test_split(data, test_size=TEST_SIZE, random_state=SEED)
print("TRAIN size:", len(df_train))
print("TEST size:", len(df_test))

NameError: name 'train_test_split' is not defined

0               awww bummer shoulda got david carr third day
1          upset update facebook texting might cry result...
2          dived many times ball managed save 50 rest go ...
3                           whole body feels itchy like fire
4                                           behaving mad see
                                 ...                        
1599995                        woke school best feeling ever
1599996             thewdb com cool hear old walt interviews
1599997                      ready mojo makeover ask details
1599998    happy 38th birthday boo alll time tupac amaru ...
1599999    happy charitytuesday thenspcc sparkscharity sp...
Name: text, Length: 1600000, dtype: object

In [43]:
test_tensor = torch.tensor(test.to_numpy())
train_tensor = torch.tensor(train.to_numpy())

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.